In [1]:
import os
import sklearn
import pandas as pd
import unidecode
import csv

In [2]:
n_gram_value = 5

In [3]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [4]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
    possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [5]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [6]:
def writeToCsv(processed_list):
    with open("output.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        for row in processed_list:
            wr.writerows(row)

In [15]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list    

In [16]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Rahul","Final")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
for file in file_list:
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)   
  

In [9]:
processed_list



[[["National Football League's American Conference", '000', 70, 74, 0],
  ["National Football League's American", '000', 70, 73, 0],
  ["Football League's American Conference", '000', 71, 74, 0],
  ['Nielsen Media Research', '000', 39, 41, 0],
  ["National Football League's", '000', 70, 72, 0],
  ["Football League's American", '000', 71, 73, 0],
  ["League's American Conference", '000', 72, 74, 0],
  ['<markup>Katz Television Group</markup>', '000', 142, 144, 1],
  ['New York A', '000', 146, 148, 0],
  ['American Idol', '000', 0, 1, 0],
  ['Nielsen Media', '000', 39, 40, 0],
  ['Media Research', '000', 40, 41, 0],
  ['National Football', '000', 70, 71, 0],
  ["Football League's", '000', 71, 72, 0],
  ["League's American", '000', 72, 73, 0],
  ['American Conference', '000', 73, 74, 0],
  ['Simon Cowell', '000', 113, 114, 0],
  ['Randy Jackson', '000', 116, 117, 0],
  ['Bill Carroll', '000', 132, 133, 0],
  ['New York', '000', 146, 147, 0],
  ['York A', '000', 147, 148, 0],
  ['American'

0

In [11]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [13]:
["markup" in word for word in ngram].count(True)==2

True